# Нормализация, секционирование и чанкинг Markdown

В этом ноутбуке выполняем три последовательных этапа обработки Markdown-документов для подготовки к векторизации и загрузке в Qdrant:

## Этапы обработки

1. **Глубокая нормализация** — очистка OCR-артефактов и приведение текста к корректному виду
2. **Секционирование** — разбиение документа на логические разделы по заголовкам
3. **Чанкинг** — деление разделов на перекрывающиеся фрагменты оптимального размера

## Пути к файлам

- **Вход (этап 1)**: `data/processed/md/{PHB,DMG}.md` — Markdown после Docling
- **Промежуточный результат**: `data/processed/md_clean/{PHB,DMG}.md` — очищенный Markdown
- **Выход (этап 3)**: `data/processed/chunks/{PHB,DMG}.jsonl` — чанки с метаданными в формате JSONL

## Параметры чанкинга

- **Размер чанка**: ~800 токенов (настраивается в `configs/ingest.yaml`)
- **Перекрытие**: 120 токенов для сохранения контекста между чанками
- **Метаданные**: каждый чанк содержит ID, название книги, главу, раздел, диапазон страниц


## Шаг 1. Нормализация Markdown (очистка шума)

Перед чанкингом очищаем Docling/OCR‑артефакты и приводим текст к «готовому к векторизации» виду. Используется функция `deep_normalize_markdown`, которая последовательно применяет 13 этапов очистки:

### Этапы нормализации (в порядке применения)

**1. Унификация и декодирование**
- Унификация переводов строк (Unix-стиль `\n`)
- Декодирование HTML-сущностей: `&amp;` → `&`, `&quot;` → `"`
- Декодирование Unicode escape-последовательностей: `/uniXXXX` → соответствующий символ

**2. Базовая нормализация**
- Исправление мягких переносов: `слово-\nпродолжение` → `словопродолжение`
- Удаление конечных пробелов
- Схлопывание избыточных пустых строк (max 2 подряд)

**3. Удаление шумных строк**
- Сбрасываем OCR-артефакты: `@DMG.md (2-11)`, короткие строки без букв

**4. Склейка разорванных слов**
- `П оследующие` → `Последующие`
- `D UNGEONS` → `DUNGEONS`
- Внутрисловные переносы строк заменяются пробелом

**5. Исправление дефисов и тире**
- Внутри слов: `a - b` → `a-b`
- Между словами: ` - ` → ` — ` (длинное тире)
- Удаление лишних пробелов перед знаками препинания

**6. Восстановление буквиц** (требуется `pymorphy2`)
- Восстанавливаем пропавшую первую букву в параграфах, начинающихся с буквицы

**7-8. Исправление омографов и гомоглифов**
- Латинские буквы → кириллица внутри русских слов: `A,B,C,E,H,K,M,O,P,T,X,Y` + строчные
- Дополнительные гомоглифы: `h→н`, `m→м`, `t→т` (только в словах с кириллицей)

**9. Короткие латинские слова в русском тексте**
- Безопасная замена: `Kak/Na/He/Ha/TOM/ACT/C` → `Как/На/Не/На/ТОМ/АСТ/С`
- При наличии `pymorphy2` — проверка валидности по словарю

**10. Замена цифр в русских словах**
- `3/6/0` → `З/з`, `Б/б`, `О/о` (с учётом регистра и позиции)
- `4` в конце слова → `й/Й`
- **Игнорируются**: нотации костей (`1к4`, `к10`, `2d6`, `+N`)

**11. Удаление артефакта 'f'**
- Латинское `f` перед русской гласной: `fа` → `а`

**12. Удаление сдвоенных слов**
- После склейки строк: `персонажи персонажи` → `персонажи`

**13. Финальная нормализация**
- Unicode NFC (каноническая композиция)
- Схлопывание множественных пробелов
- Удаление конечных пробелов и лишних пустых строк

### Результат

Очищенные файлы сохраняются в `data/processed/md_clean/*.md` и используются для чанкинга на следующем этапе.


In [ ]:
# Нормализация исходных Markdown в отдельную директорию
import sys
from pathlib import Path
import json
import difflib
from datetime import datetime


def _detect_repo_root() -> Path:
    """
    Ищет корень репозитория, поднимаясь по директориям вверх.
    Корень определяется наличием папок 'src' и 'data'.
    """
    start = Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / 'src').is_dir() and (p / 'data').is_dir():
            return p
    return start


try:
    # Сработает, если код запущен как скрипт, а не ноутбук
    repo_root = Path(__file__).resolve().parent.parent
except NameError:
    # В ноутбуке __file__ нет — ищем корень по структуре проекта
    repo_root = _detect_repo_root()

# Добавляем src/ в PYTHONPATH для импорта модулей проекта
sys.path.append(str(repo_root / 'src'))

# Импортируем функцию глубокой нормализации и путь к конфигурации
from dnd_rag.core.pipelines import normalize_md_dir_pipeline  # type: ignore
from dnd_rag.core.config import DEFAULT_CONFIG_PATH  # type: ignore

# Указываем пути к директориям
RAW_MD_DIR = repo_root / 'data' / 'processed' / 'md'  # Исходные Markdown после Docling
CLEAN_MD_DIR = repo_root / 'data' / 'processed' / 'md_clean'  # Очищенные Markdown

# Запускаем глубокую нормализацию:
# - Удаление OCR-артефактов и шумных строк
# - Склейка разорванных слов и переносов
# - Исправление омографов (латиница ↔ кириллица)
# - Восстановление буквиц
# - Замена цифр на буквы в русских словах
# - Нормализация дефисов, тире и пробелов
normalized = normalize_md_dir_pipeline(
    RAW_MD_DIR,
    CLEAN_MD_DIR,
    config_path=DEFAULT_CONFIG_PATH,  # Используется для логирования, сама нормализация не зависит от конфига
)

# Логирование изменений в logs/ и подсчёт общей суммы изменений
LOGS_DIR = repo_root / 'logs'
LOGS_DIR.mkdir(parents=True, exist_ok=True)


def _compute_changes_by_line_and_word(original_text: str, cleaned_text: str):
    """
    Возвращает список изменений с привязкой к номеру строки исходного файла.
    - Многострочные замены агрегируются как абзацы (before/after — блоки).
    - Однострочные изменения детализируются по словам (before/after — фрагменты).
    """
    # Унифицируем переводы строк и разбиваем на строки
    orig_lines = original_text.replace("\r\n", "\n").replace("\r", "\n").split("\n")
    clean_lines = cleaned_text.replace("\r\n", "\n").replace("\r", "\n").split("\n")

    changes = []
    sm = difflib.SequenceMatcher(a=orig_lines, b=clean_lines, autojunk=False)

    for tag, i1, i2, j1, j2 in sm.get_opcodes():
        if tag == "equal":
            continue

        # Если изменён блок из нескольких строк — считаем это абзацем
        if (i2 - i1) > 1 or (j2 - j1) > 1:
            before_block = "\n".join(orig_lines[i1:i2])
            after_block = "\n".join(clean_lines[j1:j2])
            line_no = i1 + 1  # 1‑based
            changes.append({
                "line": line_no,
                "before": before_block,
                "after": after_block,
            })
            continue

        # Однострочные операции (replace/delete/insert) — детализация по словам
        before_line = orig_lines[i1] if (i2 - i1) == 1 else ""
        after_line = clean_lines[j1] if (j2 - j1) == 1 else ""

        w1 = before_line.split()
        w2 = after_line.split()
        smw = difflib.SequenceMatcher(a=w1, b=w2, autojunk=False)
        line_no = (i1 + 1) if (i2 - i1) >= 1 else max(i1, 0) + 1

        for wtag, a1, a2, b1, b2 in smw.get_opcodes():
            if wtag == "equal":
                continue
            before = " ".join(w1[a1:a2]) if (a2 - a1) > 0 else ""
            after = " ".join(w2[b1:b2]) if (b2 - b1) > 0 else ""
            changes.append({
                "line": line_no,
                "before": before,
                "after": after,
            })

    return changes


# Генерация логов по всем файлам, созданным пайплайном
total_changes_all = 0
for out_file in normalized:
    out_path = Path(out_file)
    in_path = RAW_MD_DIR / out_path.name
    if not in_path.exists():
        # Если исходного файла нет, пропускаем
        continue

    original_text = in_path.read_text(encoding="utf-8")
    cleaned_text = out_path.read_text(encoding="utf-8")

    changes = _compute_changes_by_line_and_word(original_text, cleaned_text)
    total_changes_all += len(changes)

    ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    book_code = out_path.stem  # Например, DMG
    log_name = f"{ts}_pdf-processing_{book_code}.json"
    log_path = LOGS_DIR / log_name

    payload = {
        "file": out_path.name,
        "original_path": str(in_path),
        "clean_path": str(out_path),
        "created_at": ts,
        "total_changes": len(changes),
        "changes": changes,
    }

    log_path.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")

print(f"Сумма изменений: {total_changes_all}")

# Выводим список обработанных файлов
normalized

## Шаг 2. Секционирование очищенного Markdown

На этом шаге мы строим структурированное представление книги без деления текста по длине:

1. Проходим по очищенным файлам из `data/processed/md_clean`.
2. Парсим заголовки (H1/H2/H3+) и сохраняем каждую секцию со связанной иерархией в `data/processed/sections/*.jsonl`.
3. Для каждой секции фиксируем `book_title`, `chapter_title`, `section_path` и полный текст секции — именно эти данные пойдут в следующий этап чанкинга.


In [1]:
# Импорт и запуск секционирования
import sys
from pathlib import Path

def _detect_repo_root() -> Path:
    """
    Ищет корень репозитория, поднимаясь по директориям вверх.
    Корень определяется наличием папок 'src' и 'data'.
    """
    start = Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / 'src').is_dir() and (p / 'data').is_dir():
            return p
    return start

# Проверяем, определена ли переменная repo_root из предыдущей ячейки
try:
    repo_root
except NameError:
    # Если переменная не определена, пытаемся найти корень репозитория
    try:
        repo_root = Path(__file__).resolve().parents[1]
    except NameError:
        repo_root = _detect_repo_root()

# Добавляем src/ в PYTHONPATH для импорта модулей проекта
sys.path.append(str(repo_root / 'src'))

# Импортируем пайплайн секционирования
from dnd_rag.core.pipelines import sections_from_md_pipeline  # type: ignore

# Указываем пути к директориям
IN_MD_DIR = repo_root / 'data' / 'processed' / 'md_clean'   # Очищенные Markdown
OUT_SEC_DIR = repo_root / 'data' / 'processed' / 'sections'  # JSONL файлы с секциями

# Запускаем секционирование (без деления по длине)
produced = sections_from_md_pipeline(IN_MD_DIR, OUT_SEC_DIR)

# Выводим список созданных JSONL файлов с секциями
produced


c:\Users\Shchurov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-11-16 22:52:08,626 - INFO - Loading dictionaries from c:\Users\Shchurov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymorphy2_dicts_ru\data
2025-11-16 22:52:08,711 - INFO - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


[WindowsPath('C:/Users/Shchurov/Хранилище/Документы/05_Программирование/02_projects/personal/dnd_rule_assistant/data/processed/sections/DMG.jsonl')]

## Дальнейшие шаги

Для индексации и эмбеддингов используйте отдельный ноутбук `03_index_and_embedding.ipynb`:
- Шаг 1: инициализация/проверка коллекции в Qdrant (без запуска Docker — сервис уже работает)
- Шаг 2: чанкинг из `data/processed/sections` и эмбеддинг текстов с загрузкой в Qdrant
